In [1]:
from finlab.data import Data
from talib import abstract
import pandas as pd
import datetime
data = Data()
data.date = datetime.date(2020,9,18)

In [37]:
open_price = data.get('開盤價',60)
close_price = data.get('收盤價', 60)
BBAND_upper = []
BBAND_MA = []
BBAND_lower = []
for i in close_price.columns:
    upper, MA, lower = abstract.BBANDS(close_price[i], timeperiod=20, nbdevup=2.1, nbdevdn=2.1, matype=0)
    BBAND_upper.append([upper])
    BBAND_MA.append([MA])
    BBAND_lower.append([lower])

In [38]:
BBAND_upper_df = pd.DataFrame(BBAND_upper[0])
BBAND_lower_df = pd.DataFrame(BBAND_lower[0])
BBAND_MA_df = pd.DataFrame(BBAND_MA[0])
for i in range(len(BBAND_upper)-1):
    BBAND_upper_df = pd.DataFrame.append(BBAND_upper_df,BBAND_upper[i+1])
    BBAND_lower_df = pd.DataFrame.append(BBAND_lower_df,BBAND_lower[i+1])
    BBAND_MA_df = pd.DataFrame.append(BBAND_MA_df,BBAND_MA[i+1])

In [39]:
BBAND_upper_3 = []
for i in close_price.columns:
    upper, MA, lower = abstract.BBANDS(close_price[i], timeperiod=20, nbdevup=3.0, nbdevdn=2.1, matype=0)
    BBAND_upper_3.append([upper])
BBANDS_upper_df_3 = pd.DataFrame(BBAND_upper_3[0])
for i in range(len(BBAND_upper_3)-1):
    BBANDS_upper_df_3 = pd.DataFrame.append(BBANDS_upper_df_3,BBAND_upper_3[i+1])

In [40]:
BBAND_upper_df.columns = close_price.index
BBAND_upper_df.index = close_price.columns
BBAND_lower_df.columns = close_price.index
BBAND_lower_df.index = close_price.columns
BBAND_MA_df.columns = close_price.index
BBAND_MA_df.index = close_price.columns
BBAND_lower_df = BBAND_lower_df.transpose()
BBAND_upper_df = BBAND_upper_df.transpose()
BBAND_MA_df = BBAND_MA_df.transpose()
BBANDS_upper_df_3.columns = close_price.index
BBANDS_upper_df_3.index = close_price.columns
BBANDS_upper_df_3 = BBANDS_upper_df_3.transpose()
BandWidth = (BBAND_upper_df-BBAND_lower_df)/BBAND_MA_df


In [43]:
condition1 = BandWidth.iloc[-1] < 0.15
for i in range(10):
    condition1 = condition1 & (BandWidth.iloc[-2-i]<0.15)
#BandWidth < 15% for at least 10 days

stock_id
0050    True
0051    True
0052    True
0053    True
0055    True
        ... 
9940    True
9942    True
9943    True
9944    True
9946    True
Length: 601, dtype: bool

In [19]:
condition2 = (close_price.iloc[-1] > BBAND_upper_df.iloc[-1]) & (close_price.iloc[-1] > open_price.iloc[-1])
condition2_2 = close_price.iloc[-1] < BBANDS_upper_df_3.iloc[-1]
#close price break through BBAND upper and get red candlestick

In [20]:
volume = data.get('成交股數',200)
volume_average_5 = volume.rolling(5).mean()
condition3 = (volume.iloc[-1] > (volume_average_5.iloc[-1]*2)) & (volume.iloc[-1] < (volume_average_5.iloc[-1]*10))
#volumn has to be twice greater and 10-time lesser than 5-day average 
turnover = data.get('成交金額',200)
condition4 = turnover.iloc[-1] > 10000000
#turnover has to be greater than 10 milllions

In [21]:
condition5 = BBAND_MA_df.iloc[-1] > BBAND_MA_df.iloc[-2]
#均線翻揚
condition6 = close_price.iloc[-1] > close_price.iloc[-21:-2].max()
#創20天新高

In [22]:
select = condition1 & condition2 & condition3 & condition4 & condition5 & condition6 & condition2_2
select[select]

stock_id
1219    True
1795    True
1808    True
2355    True
2374    True
2485    True
2493    True
2495    True
3543    True
5285    True
6505    True
6592    True
8404    True
dtype: bool

In [1]:
def water(data):
    from talib import abstract
    import pandas as pd
    from finlab.data import Data
    open_price = data.get('開盤價',60)
    close_price = data.get('收盤價', 60)
    BBAND_upper = []
    BBAND_MA = []
    BBAND_lower = []
    
    for i in close_price.columns:
        upper, MA, lower = abstract.BBANDS(close_price[i], timeperiod=20, nbdevup=2.1, nbdevdn=2.1, matype=0)
        BBAND_upper.append([upper])
        BBAND_MA.append([MA])
        BBAND_lower.append([lower])
        
    BBAND_upper_df = pd.DataFrame(BBAND_upper[0])
    BBAND_lower_df = pd.DataFrame(BBAND_lower[0])
    BBAND_MA_df = pd.DataFrame(BBAND_MA[0])
    
    for i in range(len(BBAND_upper)-1):
        BBAND_upper_df = pd.DataFrame.append(BBAND_upper_df,BBAND_upper[i+1])
        BBAND_lower_df = pd.DataFrame.append(BBAND_lower_df,BBAND_lower[i+1])
        BBAND_MA_df = pd.DataFrame.append(BBAND_MA_df,BBAND_MA[i+1])
        
    BBAND_upper_3 = []
    for i in close_price.columns:
        upper, MA, lower = abstract.BBANDS(close_price[i], timeperiod=20, nbdevup=3.0, nbdevdn=2.1, matype=0)
        BBAND_upper_3.append([upper])
    BBANDS_upper_df_3 = pd.DataFrame(BBAND_upper_3[0])
    for i in range(len(BBAND_upper_3)-1):
        BBANDS_upper_df_3 = pd.DataFrame.append(BBANDS_upper_df_3,BBAND_upper_3[i+1])
        
    BBAND_upper_df.columns = close_price.index
    BBAND_upper_df.index = close_price.columns
    BBAND_lower_df.columns = close_price.index
    BBAND_lower_df.index = close_price.columns
    BBAND_MA_df.columns = close_price.index
    BBAND_MA_df.index = close_price.columns
    BBAND_lower_df = BBAND_lower_df.transpose()
    BBAND_upper_df = BBAND_upper_df.transpose()
    BBAND_MA_df = BBAND_MA_df.transpose()
    BBANDS_upper_df_3.columns = close_price.index
    BBANDS_upper_df_3.index = close_price.columns
    BBANDS_upper_df_3 = BBANDS_upper_df_3.transpose()

    BandWidth = (BBAND_upper_df-BBAND_lower_df)/BBAND_MA_df
    condition1 = BandWidth.iloc[-1] < 0.15
    #for i in range(30):
    #    condition1 = condition1 & (BandWidth.iloc[-2-i]<0.15)
    condition1_1 = BandWidth.iloc[-1]<0.15
    for i in range(15):
        condition1_1 = condition1_1 & (BandWidth.iloc[-2-i]<0.15)
    #BandWidth < 15% for at least 10 days
    condition2 = (close_price.iloc[-1] > BBAND_upper_df.iloc[-1]) & (close_price.iloc[-1] > open_price.iloc[-1])
    condition2_2 = close_price.iloc[-1] < BBANDS_upper_df_3.iloc[-1]
    #close price break through BBAND upper and get red candlestick
    volume = data.get('成交股數',10)
    volume_average_5 = volume.rolling(5).mean()
    condition3 = (volume.iloc[-1] > (volume_average_5.iloc[-1]*2)) & (volume.iloc[-1] < (volume_average_5.iloc[-1]*10))
    #volumn has to be twice greater and 10-time lesser than 5-day average 
    turnover = data.get('成交金額',1)
    condition4 = turnover.iloc[-1] > 10000000
    #turnover has to be greater than 10 milllions
    condition5 = BBAND_MA_df.iloc[-1] > BBAND_MA_df.iloc[-2]
    #均線翻揚
    condition6 = close_price.iloc[-1] > close_price.iloc[-21:-2].max()
    #創20天新高
    select = condition1 & condition2 & condition3 & condition4 & condition5 & condition6 & condition1_1 & condition2_2
    return select[select]

In [2]:
from finlab.data import Data
from talib import abstract
import pandas as pd
import datetime
data = Data()
data.date = datetime.date(2021,4,22)
stock = water(data)

In [3]:
stock
#data.get('收盤價',6)

stock_id
1464    True
1603    True
2536    True
2701    True
5531    True
dtype: bool

In [1]:
import pandas as pd
def mystrategy(data):
    
    close = data.get("收盤價", 120)
    condition1 = close.max() / close.min()
    
    rev = data.get("當月營收", 14)
    condition2 = (rev.iloc[-1] / rev.iloc[-13]) 
    condition3 = (rev.iloc[-2] / rev.iloc[-14])
    
    close_60_days = close.rolling(60).mean()
    condition5 = close_60_days.iloc[-1] > close_60_days.iloc[-15]
    
    稅後淨利 = data.get('本期淨利（淨損）', 5)
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 5)
    權益總額 = data.get('權益總額', 5)
    # 並且把它們合併起來
    權益總計.fillna(權益總額, inplace=True)
    roe = 稅後淨利 / 權益總計
    condition4 = roe.iloc[-1] / roe.iloc[-5]
    
    select_stock = (
        condition1.rank() + # 該數值越小越好
        condition2.rank(ascending=False) + # 該值越大越好
        condition3.rank(ascending=False) + # 該值越大越好
        condition4.rank(ascending=False)   # 該值越大越好
    ).rank() <= 10
    
    return select_stock[select_stock]

In [9]:
from finlab.data import Data
from finlab.backtest import portfolio
from talib import abstract
import pandas as pd
import datetime
data = Data()
data.date = datetime.date(2020,10,1)
stock = mystrategy(data)
p, total_invest_money = portfolio(stock.index, 2000000, data)
print('---------------')
print('|  portfolio  |')
print('---------------')
print(p)
print('total cost')
print(total_invest_money)

estimate price according to 2020-09-30 00:00:00
initial number of stock 10
after considering fee 10
after considering 1000 share 9
---------------
|  portfolio  |
---------------
stock_id
1558     1.0
2530    14.0
2536     9.0
3530     2.0
3706     7.0
4935     2.0
6277     2.0
6281     3.0
9958     1.0
Name: 2020-09-30 00:00:00, dtype: float64
total cost
1645050.0


In [44]:
from finlab.backtest_MA import backtest
from finlab.data import Data
import datetime

%matplotlib inline
data = Data()
backtest(datetime.date(2020,7,26), datetime.date(2020,8,5), water , data)

NameError: name 'water' is not defined